In [ ]:
import pandas as pd
import json
import os
from collections import defaultdict 
from matplotlib import pyplot as plt 

In [ ]:
def ns_to_s(e, name):
    if e[name] is not None:
        e[name] = e[name]  / 1000.0 / 1000.0 / 1000.0

def load_data(filename):
    name = os.path.basename(filename).split('.')[0]
    with open(filename) as f:
        entries = []
        for line in f.readlines():
            if line.startswith("{"):
                d = json.loads(line)
                e = {'bench': name, **d['conf'], **d['stats']}
                ns_to_s(e, 'time_ns')                
                entries.append(e)
    return entries
logs = ["sdram_w1.log", "sdram_w2.log", "c1.log", "i2c_k1.log"]

In [ ]:
data = []
for log in logs:
    data += load_data(log)
table = pd.DataFrame(data)

In [ ]:
table

In [ ]:
def load_time(filename):
    name = os.path.basename(filename).split('.')[0]
    with open(filename) as f:
        time_by_window_size = defaultdict(list)
        window_size_count = defaultdict(int)
        for line in f.readlines():
            if line.startswith("{"):
                d = json.loads(line)
                e = {'bench': name, **d['conf'], **d['stats']}
                window_size_count[e['window_size']] += 1
                if not e['success']:
                    continue
                ns_to_s(e, 'time_ns')
                time_by_window_size[e['window_size']].append(e['time_ns'])
    return time_by_window_size, window_size_count

In [ ]:
def plot_window_size_vs_time(filename):
    time_by_window_size, window_size_count = load_time(filename)
    max_window_size = max(time_by_window_size.keys()) - 1 # make sure we do not look at incomplete data
    x = sorted(list(time_by_window_size.keys()))[:-1]
    y = [sum(time_by_window_size[w]) for w in x]
    print(f"min window size: {x[0]}, {len(time_by_window_size[x[0]])} / {window_size_count[x[0]]}")
    plt.plot(x, y)
    success_y = [float(len(time_by_window_size[w])) /window_size_count[w]  for w in x]
    

In [ ]:
plot_window_size_vs_time('i2c_k1.log')
plot_window_size_vs_time('sdram_w2.log')
plot_window_size_vs_time('c1.log')
plt.xlabel("window size (cycles)")
plt.ylabel("average time (s)")
plt.savefig("window_size" + ".png")
plt.savefig("window_size" + ".svg")
plt.show()

In [ ]:
def load_unroll_time(filename):
    name = os.path.basename(filename).split('.')[0]
    with open(filename) as f:
        wrong_time = []
        right_time = []
        for line in f.readlines():
            if line.startswith("==="):
                break
            if line.startswith("{"):
                d = json.loads(line)
                ns_to_s(d, 'time_ns')
                x, y = d['end_step'], d['time_ns']
                if d['success']:
                    right_time += [(x, y)]
                else:
                    wrong_time += [(x, y)]
    return wrong_time, right_time

def plot_unrolling_vs_time(filename, error_at: int):
    wrong_time, right_time = load_unroll_time(filename)
    wrong_time = [(x,y) for x,y in wrong_time if x >= error_at]
    right_time = [(x,y) for x,y in right_time if x >= error_at]
    x = [x for x,_ in wrong_time] + [x for x,_ in right_time]
    #plt.plot(*zip(*sorted(wrong_time + right_time)))
    first_success = min(x for x,_ in right_time)
    print(f"first success at {first_success}")
    #plt.plot(*zip(*wrong_time), 'r')
    #plt.plot(*zip(*right_time), 'g')
    plt.plot(*zip(*sorted(right_time + wrong_time)))
    
    plt.title(filename)
    plt.xlabel("window size")
    plt.ylabel("average time (s)")
    plt.savefig(filename + ".png")
    plt.show()

In [ ]:
plot_unrolling_vs_time("i2c_k1_unroll.log", error_at=1238)
plot_unrolling_vs_time("sdram_w2_unroll.log", error_at=130)
# never succeeds
# plot_unrolling_vs_time("c1_unroll.log")